### YOLOv5 installation


### As long as you change the *path_to_project* variable to your path up until the project and *project_root* NATO-Symbols part to the project folder name, code should run seamlessly.

In [7]:
#from google.colab import drive
#drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
#!cd /content
#!rm -rf yolov5
import os

path_to_project=r"C:\Users\karlk\PycharmProjects"
project_root=path_to_project+r"\NATO_symbols\YOLO"
yolo_dir=project_root+r"\yolov5"
#NOTE THAT YOLO EXPECTS THE datasets FOLDER TO BE IN THE SAME ROOT FOLDER AS IT IS
img_dir=project_root+r"\datasets\images"
label_dir=project_root+r"\datasets\labels"
if yolo_dir==None:
  yolo_dir=input("Please paste the directory you want to install YOLOv5 into. WITHOUT / IN THE END")

#%cd {project_root}

#![ -d {yolo_dir} ] && echo "YOLO DIRECTORY EXISTS SKIPPING INSTALL"
#![ ! -d {yolo_dir} ] && echo "INSTALLING YOLOv5 INTO DESIGNATED DIRECTORY" && git clone https://github.com/ultralytics/yolov5


### Requirements installation
NB! Meant to be run in Google Colab.

In [17]:
import sys
if project_root+r"\scripts" not in sys.path:
  sys.path.append(project_root+r"\scripts")
yolo_reqs=yolo_dir+r"\requirements.txt"
project_reqs=project_root+r"\requirements.txt"
!pip install wandb
!pip install -r  {yolo_reqs}
!pip install -r {project_reqs}

In [12]:
#!cp /content/drive/MyDrive/Yolov5/NATO-Symbols/scripts/augment_tools.py /content #THIS IS ONLY NEEDED IF YOU OTHERWISE GET AN IMPORT ERROR

### Generate a YOLOv5 dataset

This is based on single-symbol images that will be randomly placed on a white canvas to compose a set of multi-symbol images that will be used in training the YOLOv5 weights.

In [18]:
import numpy as np
import cv2
from random import randint
from scipy import ndimage
import os
import re
from tqdm import tqdm
from augment_tools import augment
from matplotlib import pylab as plt
try:
  from google.colab.patches import cv2_imshow
except:
  pass
clean=False
dataset_sz=450
dim=(720,1280)
label_list=["advance_to_contact", "attack", "block", "counterattack", "cover", "delay", "destroy", "disrupt", "guard", "occupy", "retain", "screen", "secure", "seize", "support_by_fire"]
localize_list=["symbol"]

In [19]:
raws=[]
raw_labels=[]
datasets_root=project_root+r"\datasets"
raw_clean_dir=datasets_root+r"\train_data"
raw_dir=datasets_root+r"\train_data"

if clean:
  file_list=os.listdir(raw_clean_dir)
  print(file_list)

  for title in file_list:
    raw=cv2.imread(raw_clean_dir+"/"+title, cv2.IMREAD_GRAYSCALE)
    raw=cv2.resize(raw, (150,150), cv2.INTER_AREA)
    raw=cv2.adaptiveThreshold(raw, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 19)

    raw=raw[np.argwhere(np.amin(raw,axis=1)<120)[0][0]:np.argwhere(np.amin(raw,axis=1)<120)[-1][0],:]
    raw = raw[:,np.argwhere(np.amin(raw,axis=0) < 120)[0][0]:np.argwhere(np.amin(raw,axis=0) < 120)[-1][0]]

    raws.append(raw)
    raw_labels.append(re.findall('([a-zA-Z_ ]*)\d*.*', title)[0])

  cv2_imshow(raw)

else:
  #for subdir in os.listdir(raw_dir):
  dir=raw_dir #+ '\\' + subdir
  file_list=os.listdir(dir)
  print("Checking directory ", dir)

  for title in file_list:
    raw=cv2.imread(dir+"/"+title, cv2.IMREAD_GRAYSCALE)
    raw=cv2.resize(raw, (150,150), cv2.INTER_AREA)
    raw=cv2.adaptiveThreshold(raw, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 19)

    raw=raw[np.argwhere(np.amin(raw,axis=1)<120)[0][0]:np.argwhere(np.amin(raw,axis=1)<120)[-1][0],:]
    raw = raw[:,np.argwhere(np.amin(raw,axis=0) < 120)[0][0]:np.argwhere(np.amin(raw,axis=0) < 120)[-1][0]]

    raws.append(raw)
    raw_labels.append(re.findall('([a-zA-Z_ ]*)\d*.*', title)[0])
    #cv2_imshow(raw)
print("There are {} labels: {}".format(len(np.unique(raw_labels)), np.unique(raw_labels)))

Checking directory  C:\Users\karlk\PycharmProjects\NATO_symbols\YOLO\datasets\train_data
There are 15 labels: ['advance_to_contact' 'attack' 'block' 'counterattack' 'cover' 'delay'
 'destroy' 'disrupt' 'guard' 'occupy' 'retain' 'screen' 'secure' 'seize'
 'support_by_fire']


In [20]:
def generate_image(raw, raw_labels, dim=(850,850),min_sym=5,max_sym=9, max_overlap=15, bg_dim=0.85):
  canvas=np.full(dim,int(bg_dim*255))
  sym_data=[]
  locations=[]

  for i in range(randint(min_sym,max_sym)):
    choice_idx=randint(0,len(raw)-1)
    chosen=np.copy(raw[choice_idx])
    chosen=augment(chosen.astype(np.uint8), apply_resize=True, apply_flip=True, apply_rotation=True, apply_transformation=True, apply_boldness=True)

    while(True):
      overlap=False
      top_leftX=randint(0,dim[0]-chosen.shape[0])
      top_leftY=randint(0,dim[1]-chosen.shape[1])
      for j in range(len(locations)):
        if (top_leftX < locations[j][1][0]-max_overlap) and (top_leftY < locations[j][1][1]-max_overlap) and (top_leftX+chosen.shape[0] > locations[j][0][0]+max_overlap) and (top_leftY+chosen.shape[1] > locations[j][0][1]+max_overlap):
          overlap=True
          break
      if not overlap:
        break

    locations.append([[top_leftX,top_leftY],[top_leftX+chosen.shape[0],top_leftY+chosen.shape[1]]])
    canvas[top_leftX:(top_leftX+chosen.shape[0]),top_leftY:(top_leftY+chosen.shape[1])][chosen < 140] = chosen[chosen < 140]
    
    #YOLO xywh format, flip x and y and w and h from previous step
    dimX, dimY = dim
    y=round(((top_leftX+(chosen.shape[0]/2))/dimX),2)
    x=round(((top_leftY+(chosen.shape[1]/2))/dimY),2)
    h=round((chosen.shape[0]/dimX),2)
    w=round((chosen.shape[1]/dimY),2)

    #Class nr
    label=raw_labels[choice_idx]
    if label=="counterattack_by_fire":
      label="counterattack"
    class_nr=label_list.index(label)
    sym_data.append([class_nr, x, y, w, h])


  return canvas, sym_data

In [21]:
if not os.path.exists(img_dir+"/train/img0.jpg"):
  trainnr=int(0.85*dataset_sz)

  for i in tqdm(range(dataset_sz)):
    img, img_data = generate_image(raws, raw_labels, dim)

    if i<trainnr:
      if cv2.imwrite(img_dir+r"\train\img"+str(i)+".jpg", img):
        with open(label_dir+r"\train\img"+str(i)+".txt", 'w') as f:
          for j in img_data:
            for k in j:
              f.write(str(k)+" ")
            f.write("\n")
    else:
      if cv2.imwrite(img_dir+r"\val\img"+str(i)+".jpg", img):
        with open(label_dir+r"\val\img"+str(i)+".txt", 'w') as f:
          for j in img_data:
            for k in j:
              f.write(str(k)+" ")
            f.write("\n")


### Training YOLOv5

In [22]:
import wandb
train_dir=yolo_dir+r"\train.py"
detect_dir=yolo_dir+r"\detect.py"

In [23]:
wandb.login()

wandb: Currently logged in as: karlkk. Use `wandb login --relogin` to force relogin


True

In [32]:
run_id = input("What is the name of the run? Be descriptive. Example 5l-300-colabGPU. ") #5l-300-colabGPU
if not run_id:
  run_id = "unnamed"
!python {train_dir} --img 640 --batch 32 --epochs 3 --data NATO-Symbols.yaml --weights yolov5m.pt --project NATO-Symbols-Log --name {run_id} --cache

github: skipping check (not a git repository), for updates see https://github.com/ultralytics/yolov5

wandb: Currently logged in as: karlkk. Use `wandb login --relogin` to force relogin
train: weights=yolov5m.pt, cfg=, data=NATO-Symbols.yaml, hyp=yolov5\data\hyps\hyp.scratch-low.yaml, epochs=3, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=NATO-Symbols-Log, name=5l-3-selfc, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
YOLOv5  2022-5-24 Python-3.9.7 torch-1.11.0+cpu CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, 


val: Scanning 'C:\Users\karlk\PycharmProjects\NATO_symbols\YOLO\datasets\labels\val' images and labels...68 found, 0 missing, 0 empty, 0 corrupt: 100%|##########| 68/68 [00:05<00:00, 13.42it/s]
val: New cache created: C:\Users\karlk\PycharmProjects\NATO_symbols\YOLO\datasets\labels\val.cache

val: Caching images (0.0GB ram): 100%|##########| 68/68 [00:00<00:00, 809.33it/s]
wandb: Currently logged in as: karlkk. Use `wandb login --relogin` to force relogin
wandb: Currently logged in as: karlkk. Use `wandb login --relogin` to force relogin
wandb: Currently logged in as: karlkk. Use `wandb login --relogin` to force relogin
wandb: Currently logged in as: karlkk. Use `wandb login --relogin` to force relogin
wandb: Currently logged in as: karlkk. Use `wandb login --relogin` to force relogin
wandb: Currently logged in as: karlkk. Use `wandb login --relogin` to force relogin
wandb: Currently logged in as: karlkk. Use `wandb login --relogin` to force relogin
wandb: Currently logged in as: karl

# Detection

Test the best achieved weights on a set of images.

In [44]:
run_id = "5m_300_colabGPU"

In [45]:
weights_dir=project_root+"/NATO-Symbols-Log/"+run_id+"/weights/best.pt"
detect_dir=yolo_dir+"/detect.py"
src_dir=project_root+"/datasets/test" ## multiply symbols pictures should be used here

In [46]:
#Make all the test images greyscale too to compare results
for title in os.listdir(src_dir):
    img=cv2.imread(src_dir+r"\\"+title, cv2.IMREAD_GRAYSCALE)
    img=cv2.adaptiveThreshold(img, 200, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 301, 110)
    cv2.imwrite(project_root + r"\datasets\gray_test_data\\"+title, img)

src_dir_gray = project_root + r"\datasets\gray_test_data"

In [47]:
#!python /content/drive/MyDrive/Yolov5/NATO-Symbols/yolov5/detect.py --weights /content/drive/MyDrive/Yolov5/NATO-Symbols/NATO-Symbols-Log/5l-100/weights/best.pt --source /content/drive/MyDrive/Yolov5/NATO-Symbols/datasets/raw-test --conf-thres 0.8

!python {detect_dir} --weights {weights_dir} --source {src_dir_gray} --conf-thres 0.8

detect: weights=['C:\\Users\\karlk\\PycharmProjects\\NATO_symbols\\YOLO/NATO-Symbols-Log/5m_300_colabGPU/weights/best.pt'], source=C:\Users\karlk\PycharmProjects\NATO_symbols\YOLO\datasets\gray_test_data, data=yolov5\data\coco128.yaml, imgsz=[640, 640], conf_thres=0.8, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5\runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5  2022-5-24 Python-3.9.7 torch-1.11.0+cpu CPU

Traceback (most recent call last):
  File "C:\Users\karlk\PycharmProjects\NATO_symbols\YOLO\yolov5\detect.py", line 252, in <module>
    main(opt)
  File "C:\Users\karlk\PycharmProjects\NATO_symbols\YOLO\yolov5\detect.py", line 247, in main
    run(**vars(opt))
  File "C:\Users\karlk\anaconda3\lib\site-packages\torch\autograd\grad_mode.py", line 

# Hyperparameter evolution

Since YOLO has ~25 hyperparameters used for training settings, hyperparameter evolution is a very good way to finetune a model. However, this is a super time and resource expensive process. Here it was attempted to be done for 300 generations of 10 epochs, but due to Colab constraints it was more like 100 generations of 10 epochs.

In [ ]:
run_id = input("What is the name of the run? Be descriptive. Example 5l-300-colabGPU. ") #5l-300-colabGPU
if not run_id:
  run_id = "unnamed"
!python3 {train_dir} --epochs 10 --data NATO-Symbols.yaml --weights yolov5m.pt --project NATO-Symbols-Log --name {run_id} --cache --evolve